In [1]:
import copy
import numpy as np
import scipy.sparse as sp

import glpk
import gurobipy as gp
from gurobipy import GRB
from scipy.optimize import linprog
from scipy.linalg import block_diag

import IPython
# import polytope as pc

from StarV.set.sparsestar import SparseStar
from StarV.set.star import Star

In [2]:
dim = 4
lb = -np.random.rand(dim)
ub = np.random.rand(dim)
# ub[3] = lb[3]
S = SparseStar(lb, ub)
print(S)

SparseStar Set:
A: [[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
C: 
d: []
pred_lb: [-0.75365 -0.80818 -0.77602 -0.13938]
pred_ub: [0.2642  0.22015 0.01055 0.71974]
pred_depth: [0. 0. 0. 0.]
dim: 4
nVars: 4
nZVars: 0




In [3]:
W = np.random.rand(dim, dim)
b = np.random.rand(dim)
Sa = S.affineMap(W, b)
print(Sa)

SparseStar Set:
A: [[0.03139 0.73629 0.89552 0.80928 0.41272]
 [0.71934 0.30112 0.71815 0.66436 0.26604]
 [0.18789 0.34352 0.75229 0.28119 0.95945]
 [0.98222 0.92596 0.64224 0.24055 0.38035]]
C: 
d: []
pred_lb: [-0.75365 -0.80818 -0.77602 -0.13938]
pred_ub: [0.2642  0.22015 0.01055 0.71974]
pred_depth: [0. 0. 0. 0.]
dim: 4
nVars: 4
nZVars: 0




In [4]:
lb, ub = Sa.estimateRanges()
print('lb: ', lb)
print('ub: ', ub)

lb:  [-1.9328  -0.64063 -1.03092 -0.47436]
ub:  [0.72865 1.15549 1.13778 1.64455]


In [5]:
lb, ub = Sa.estimateRange([1,3])
print('lb: ', lb)
print('ub: ', ub)

lb:  [-0.64063 -0.47436]
ub:  [1.15549 1.64455]


In [6]:
print('getMin: ')
for i in range(Sa.dim):
    print(Sa.getMin(i))
print('getMax:')
for i in range(Sa.dim):
    print(Sa.getMax(i))
print('getMins: ')
print(Sa.getMins(np.arange(Sa.dim)))
print('getMaxs:')
print(Sa.getMaxs(np.arange(Sa.dim)))

getMin: 
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-07
-1.9328000415639435
-0.6406344460872003
-1.030924741382605
-0.4743647026161938
getMax:
0.7286531400632414
1.155489029234502
1.1377826696799676
1.6445478595428162
getMins: 
[-1.9328  -0.64063 -1.03092 -0.47436]
getMaxs:
[0.72865 1.15549 1.13778 1.64455]


In [7]:
Sr = 

print('getRange estimate')
print(Sa.getRange(3, 'estimate'))
print('getRange gurobi')
print(Sa.getRange(3, 'gurobi'))
print('getRange glpk')
print(Sa.getRange(3, 'glpk'))
print('getRange linprog')
print(Sa.getRange(3, 'linprog'))

getRange estimate
(-0.4743647026161938, 1.6445478595428165)
getRange gurobi
(-0.4743647026161938, 1.6445478595428162)
getRange glpk
(-0.4743647026161938, 1.6445478595428162)
getRange linprog
(-0.4743647026161938, 1.6445478595428162)


In [8]:
print('getRanges estimate')
print(np.column_stack(Sa.getRanges('estimate')))
print('getRange gurobi')
print(np.column_stack(Sa.getRanges('gurobi')))
print('getRange glpk')
print(np.column_stack(Sa.getRanges('glpk')))
print('getRange linprog')
print(np.column_stack(Sa.getRanges('linprog')))
print('getRange gurobi with relaxFactor')
print(np.column_stack(Sa.getRanges('gurobi', 0.5)))

getRanges estimate
[[-1.9328   0.72865]
 [-0.64063  1.15549]
 [-1.03092  1.13778]
 [-0.47436  1.64455]]
getRange gurobi
[[-1.9328   0.72865]
 [-0.64063  1.15549]
 [-1.03092  1.13778]
 [-0.47436  1.64455]]
getRange glpk
[[-1.9328   0.72865]
 [-0.64063  1.15549]
 [-1.03092  1.13778]
 [-0.47436  1.64455]]
getRange linprog
[[-1.9328   0.72865]
 [-0.64063  1.15549]
 [-1.03092  1.13778]
 [-0.47436  1.64455]]
getRange gurobi with relaxFactor
[[-1.9328   0.72865]
 [-0.64063  1.15549]
 [-1.03092  1.13778]
 [-0.47436  1.64455]]


In [13]:
l = Sa.getMins(np.arange(Sa.dim))
u = Sa.getMaxs(np.arange(Sa.dim))
print("l: ", l)
print("u: ", u)

map1 = np.where(l == u)[0]
print(map1)
map1 = np.where((l >= 0) & (l != u))[0]
print(map1)
map1 = np.where((u <= 0) & (l != u))[0]
print(map1)
map1 = np.where((l < 0) & (u > 0))[0]
print(map1)

l:  [-1.9328  -0.64063 -1.03092 -0.47436]
u:  [0.72865 1.15549 1.13778 1.64455]
[]
[]
[]
[0 1 2 3]


In [10]:
print(np.argsort(l)[::-1])

[3 1 2 0]


In [14]:
np.zeros(4)

array([0., 0., 0., 0.])

In [19]:
d = np.minimum(l, u)

In [20]:
np.diag(d)

array([[-1.9328 ,  0.     ,  0.     ,  0.     ],
       [ 0.     , -0.64063,  0.     ,  0.     ],
       [ 0.     ,  0.     , -1.03092,  0.     ],
       [ 0.     ,  0.     ,  0.     , -0.47436]])

In [22]:
l*u

array([-1.40834, -0.74025, -1.17297, -0.78012])

In [25]:
np.where(l == u)

(array([], dtype=int64),)

In [26]:
sp.array([0, 1, 2, 0])

AttributeError: module 'scipy.sparse' has no attribute 'array'